In [1]:
from qiskit import *
import numpy as np
import cowsay
import time
from IPython.display import clear_output

#### Random Number Generator using Hadamard

In [2]:
def random_num(bits): # Uses Hadamard
    ''' Returns a random number between 0-2^bits-1 
    '''
    qc = QuantumCircuit(bits) 
    qc.reset(0)
    qc.barrier()
    
    for i in range(bits):
        qc.h(i)

    qc.barrier()
    
    qc.measure_all()
    qc.draw('mpl')
    backend = Aer.get_backend('aer_simulator')
    job = backend.run(qc, shots=1, memory=True)
    output = job.result().get_memory()[0]
    
    return int(output, 2)

In [3]:
out = random_num(5)
print("Output: ", out)
print("The number is: ", out)

Output:  14
The number is:  14


In [4]:
# Random Number generator using h, x, and y gates
# Written by Lee
def cpu_pick_random_num_of_sticks(bits):
    ''' Returns a random number between 0-2^bits-1 
    '''
    qc = QuantumCircuit(bits) 
    qc.reset(0)
    qc.barrier()
    
    for i in range(bits):
        qc.h(i) # Hadamard (H)
        qc.x(i) # X gate
        qc.y(i) # Y gate

    qc.barrier()
    
    qc.measure_all()
    qc.draw('mpl')
    backend = Aer.get_backend('aer_simulator')
    job = backend.run(qc, shots=1, memory=True)
    output = job.result().get_memory()[0]
    
    return int(output, 2)

In [5]:
# Testing my function - it returns values from 0 to 3 (0 to 4 if )
out = cpu_pick_random_num_of_sticks(2) + 1
print("Output: ", out)
print("The number is: ", out)

Output:  3
The number is:  3


#### Quantum Sticks Class

In [6]:
class QuantumSticks:
    
    def __init__(self):
        self.turn = True # can make this random
        self.welcome()
        self.level = 'e'
        self.n = self.get_input()
        self.qc = QuantumCircuit(self.n) 
        self.initialize_sticks()
        self.output_sticks()
        self.play()
    
    
    def clear(self):
        time.sleep(1)
        clear_output(wait=True)

    def sleep(self, t):
        time.sleep(t)
        
    def welcome(self):
        cowsay.cow('Welcome to Quantum Sticks')
        
    def slow_display(self, msg):
        for x in range (0,4):  
            b = msg + "." * x
            print (b, end="\r")
            self.sleep(0.8)
        print ("\n")
    
    
   
    def get_input(self):
        print("Enter the number of sticks you want to begin the game with: ")
        ip = int(input())
        print("\nSweet!")
        print("\nSelect level [E/e]: easy, [M/m]: medium, [H/h]: hard")
        level = input().lower()
        self.level = level
        if level != 'e' and level !='m' and level != 'h':
            cowsay.cow('Damn! You dared to break the rules!\n The Quantum Gods have chosen the Hard level for you!')
            self.level = 'h'
            
        print("Starting your game at level ", self.level, "\n")
        self.slow_display("Loading")
        self.clear()
        
        bits = ip.bit_length()
        m = random_num(bits) + 1
        msg = ""
        if m != ip:
            if m == 0 or m == 1:
                m = 2
            a = 'Uh oh, the Quantum Gods have decided to go with a different number of sticks. \n You will play with '
            b = ' stick(s).\nWe wish you good luck!'
            msg = f"{a}{m}{b}"
        else:
            msg = 'And, here are your ', m , ' sticks. We wish you good luck!'
        cowsay.pig(msg)
        self.sleep(1)
        self.slow_display("Get ready")
        return m
        
    def initialize_sticks(self):
        ''' Create as many qubits as the number of sticks ==> this code might get changed
        '''
        self.qc.reset(0)
        for i in range(self.n):
            self.qc.h(i)
        
        
        
    def output_sticks(self):
        msg= f"{'Sticks remaining: '}{self.n}"
        cowsay.cow(msg)
        
    def computersTurn(self):
        self.sleep(0.5)
        for x in range (0,4):  
            b = "Hang on, the computer's thinking" + "." * x
            print (b, end="\r")
            self.sleep(0.5)
        print ("\n")
        numSticks = self.n
        
        # TODO - Replace this section with gate operations
        # Reference the section where the game initializes 
        sticks_to_pick = cpu_pick_random_num_of_sticks(2) + 1 # Code may return 0, so +1 negates this
        # This will allow the computer to pick 1, 2, 3, or 4 sticks, but it cannot pick up more than n sticks
        if sticks_to_pick >= numSticks:
            diff = sticks_to_pick - numSticks
            if diff > 0:
                sticks_to_pick = numSticks - diff
            else:
                sticks_to_pick = numSticks - 1
                
        numSticks = numSticks - sticks_to_pick
        print("\nRNG HXY: Computer Takes {} sticks".format(sticks_to_pick))
        
        #print("Testing the random number generator with 2 bits: ", testing)
        #print('\n')
#         if numSticks%4 == 3:
#             numSticks = numSticks-2
#             print("\nComputer Takes 2 sticks")
#         elif numSticks%4 == 2:
#             numSticks = numSticks-1
#             print("\nComputer Takes 1 stick")
#         elif numSticks%4 == 1:
#             numSticks = numSticks-1
#             print("\nComputer Takes 1 stick")
#         elif numSticks%4 == 0:
#             numSticks = numSticks - 3
#             print("\nComputer Takes 3 sticks")
        # End section
        
        # If there is only 1 stick left after the computer picks a stick, the player loses.
        if numSticks == 1:
            cowsay.pig("Only 1 stick left. You lose!\nGame over, buddy.")
            return 0
        
        # Game continues, return the number of sticks
        return numSticks

    def playersTurn(self):
        numSticks = self.n
        if numSticks == 0: 
            print("\nStart a new game")
            return 0
        
        print("How many sticks do you wanna pick? 1/2/3")
        pick = int(input())
        #p = self.evaluate_sticks(pick)
        #pick = p
        print("\nPlayer picks ", pick, " sticks")
        
        numSticks = numSticks - pick
        if numSticks == 1:
            cowsay.pig("Only 1 stick left. You WIN!\nGame over.")
            return 0
        return numSticks
                      
    def evaluate_sticks(self, picks):
        print("Number of sticks you wanna pick ", picks)
        # to-do: randomize the action
        # change the value of picks
        a = 'Sorry, I will have to pick '
        b = ' sticks on your behalf. \nThe Gods are erratic.'
        msg = f"{a}{picks}{b}"
        cowsay.pig(msg)
        return picks
                      
    def play(self):
        while self.n > 0 :
            if self.turn:
                #player's turn
                self.n = self.playersTurn()
                self.turn = False
            else:
                #computer's turn
                self.n = self.computersTurn()
                self.turn = True
            self.sleep(0.8)
            self.clear()
            if self.n > 0:
                self.output_sticks()
        

In [7]:
q = QuantumSticks()

  _________________________
| Welcome to Quantum Sticks |
                         \
                          \
                            ^__^
                            (oo)\_______
                            (__)\       )\/\
                                ||----w |
                                ||     ||
Enter the number of sticks you want to begin the game with: 


KeyboardInterrupt: Interrupted by user

In [ ]:
3